In [ ]:
import sys
import emission.public.pull_and_load_public_data as plpd
import pandas as pd
import emission.storage.timeseries.aggregate_timeseries as estag
public_ids = estag.TEST_PHONE_IDS
import arrow
from uuid import UUID
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

import emission.core.get_database as edb
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import numpy as np
import emission.analysis.point_features as pf
import attrdict as ad
import scipy.interpolate as spi
import emission.core.wrapper.entry as ecwe
import emission.core.wrapper.location as ecwl
pd.options.display.float_format = '{:,.10f}'.format

# Retreiving Data Between Specific Timestamps 

Get starting timestamp

In [ ]:
from_ts = arrow.get('2017-01-26T17:56:20.000-08:00').timestamp
from_ts
arrow.get(from_ts).to("America/Los_Angeles")

Get end timestamp

In [ ]:
end_ts = arrow.get("2017-01-26T18:15:00.000-08.00").timestamp
end_ts = end_ts - 290
arrow.get(end_ts)

Loading data, between timestamps, from server.

In [ ]:
plpd.request_batched_data("https://e-mission.eecs.berkeley.edu",
                          from_ts ,end_ts, map(str, public_ids))

# Organizing data for each phone into dataframes

Function to get range of points for a specific phone time series

In [ ]:
def get_points_from_ts_for_range(ts, start_ts, end_ts):
    return ts.get_data_df("background/location", 
                          estt.TimeQuery('metadata.write_ts', 
                                         start_ts, end_ts))

In [ ]:
iphone_id_3 = [UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a")]
iphone_id_4 = [UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
android_id_3 = [UUID("86842c35-da28-32ed-a90e-2da6663c5c73")]
android_id_4 = [UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

The 4 different phones and their IDs

In [ ]:
iphone_ts_3 = map(lambda u: esta.TimeSeries.get_time_series(u),
                  iphone_id_3)
iphone_ts_4 = map(lambda u: esta.TimeSeries.get_time_series(u),
                  iphone_id_4)
android_ts_3 = map(lambda u: esta.TimeSeries.get_time_series(u),
                   android_id_3)
android_ts_4 = map(lambda u: esta.TimeSeries.get_time_series(u),
                   android_id_4)

Creating the dataframes, consisting of timeseries generated above. 

In [ ]:
iphone_df_3 = get_points_from_ts_for_range(iphone_ts_3[0],from_ts,
                                           end_ts)
iphone_df_4 = get_points_from_ts_for_range(iphone_ts_4[0],from_ts,
                                           end_ts)
android_df_3 = get_points_from_ts_for_range(android_ts_3[0],from_ts,
                                            end_ts)  
android_df_4 = get_points_from_ts_for_range(android_ts_4[0],from_ts,
                                            end_ts)

Revised dataframes consisting only of latitudes, longitudes and timestamps.

In [ ]:
new_iphone_df_3 = iphone_df_3[['latitude','longitude','ts']].copy()
new_iphone_df_4 = iphone_df_4[['latitude','longitude','ts']].copy()
new_android_df_3 = android_df_3[['latitude','longitude','ts']].copy()
new_android_df_4 = android_df_4[['latitude','longitude','ts']].copy()

# Importing Ground Truth points from .CSV file

In [ ]:
ground_truth = pd.read_csv("/Users/jlamba/Desktop/e-mission berkeley trip ground truth - Sheet1 .csv")

Plotting points to see shape/ ensure it will look similar to shapes we generate for raw data.

In [ ]:
gt_time = ground_truth['fmt_time']
gt_time2 = []
for elem in gt_time:
    gt_time2.append(arrow.get('2017-01-26T'+ elem +'.000-08:00').timestamp)
ground_truth['ts'] = gt_time2
ground_truth
ground_truth.plot(x = 'ts',y = 'latitude')

# Plotting Raw Latitudes and Longitudes of 4 phones against ground truth

Plot comparing raw latitudes of all 4 phones and ground truth

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)

new_iphone_df_3.index = new_iphone_df_3['ts']
new_iphone_df_4.index = new_iphone_df_4['ts']
new_android_df_3.index = new_android_df_3['ts']
new_android_df_4.index = new_android_df_4['ts']

df1 = pd.DataFrame(new_iphone_df_3, index=new_iphone_df_3.index,columns=['latitude'])
df2 = pd.DataFrame(new_iphone_df_4, index=new_iphone_df_4.index,columns=['latitude'])
df3 = pd.DataFrame(new_android_df_3, index=new_android_df_3.index,columns=['latitude'])
df4 = pd.DataFrame(new_android_df_4, index=new_android_df_4.index,columns=['latitude'])

df1.plot(ax=axes, title = 'iphone3')
df2.plot(ax=axes,title = 'iphone4' )
df3.plot(ax=axes, label = 'android3')
df4.plot(ax = axes, label = 'android4')
ground_truth.plot(x = 'ts', y = 'latitude', ax = axes)

lines, labels = axes.get_legend_handles_labels()
print lines, labels
axes.legend(lines, ["iphone3", "iphone4", "android3", "android4","ground_truth"], loc='best') 

Plot comparing raw longitudes of all 4 phones and ground truth.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)

df2_2 = pd.DataFrame(new_iphone_df_4, index=new_iphone_df_4.index,columns=['longitude'])
df3_3 = pd.DataFrame(new_android_df_3, index=new_android_df_3.index,columns=['longitude'])
df4_4 = pd.DataFrame(new_android_df_4, index=new_android_df_4.index,columns=['longitude'])
df1_1 = pd.DataFrame(new_iphone_df_3, index=new_iphone_df_3.index,columns=['longitude'])

df1_1.plot(ax=axes)
df2_2.plot(ax=axes)
df3_3.plot(ax=axes)
df4_4.plot(ax = axes)
ground_truth.plot(x = 'ts', y = 'longitude', ax = axes)
lines, labels = axes.get_legend_handles_labels()
print lines, labels
axes.legend(lines, ["iphone3", "iphone4", "android3", "android4", 'ground truth'], loc='best')

# Interpolation

In [ ]:
import scipy.interpolate as spi
import numpy as np
%matplotlib inline

The following function takes in the 4 different phone timeseries and the ground_truth dataframe and the start and end timestampts and essentially creates interpolation functions for the latitude and longitude of each. These functions are then applied to a periodic range of timestamps and put together in a dataframe. The dataframe also contains the differences in distances between the different phones ( 4 differences to be exact), the mean values of the latitudes and longitudes, and the difference in distances between the mean values and the ground truth values. 

In [ ]:
def get_interp_distance_df(android3_ts, android4_ts, ios3_ts, ios4_ts, ground_truth_df, start_ts, end_ts):
    android3_df = android3_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    android4_df = android4_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    ios3_df = ios3_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    ios4_df = ios4_ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))
    android3_latf = spi.interp1d(x=android3_df.ts, y=android3_df.latitude, bounds_error=False,
                                fill_value="extrapolate")
    android3_lngf = spi.interp1d(x=android3_df.ts, y=android3_df.longitude, bounds_error=False, 
                                fill_value="extrapolate")
    android4_latf = spi.interp1d(x=android4_df.ts, y=android4_df.latitude, bounds_error=False, 
                                fill_value="extrapolate")
    android4_lngf = spi.interp1d(x=android4_df.ts, y=android4_df.longitude, bounds_error=False, 
                                fill_value="extrapolate")
    ios3_latf = spi.interp1d(x=ios3_df.ts, y=ios3_df.latitude, bounds_error=False, 
                            fill_value=(ios3_df.latitude.iloc[0], ios3_df.latitude.iloc[-1]))
    ios3_lngf = spi.interp1d(x=ios3_df.ts, y=ios3_df.longitude, bounds_error=False, 
                            fill_value=(ios3_df.longitude.iloc[0], ios3_df.longitude.iloc[-1]))
    ios4_latf = spi.interp1d(x=ios4_df.ts, y=ios4_df.latitude, bounds_error=False, 
                            fill_value=(ios4_df.latitude.iloc[0], ios4_df.latitude.iloc[-1]))
    ios4_lngf = spi.interp1d(x=ios4_df.ts, y=ios4_df.longitude, bounds_error=False, 
                            fill_value=(ios4_df.longitude.iloc[0], ios4_df.longitude.iloc[-1]))
    ground_truth_latf = spi.interp1d(x = ground_truth_df.ts, y = ground_truth.latitude, bounds_error=False, 
                                     fill_value=(ground_truth.latitude.iloc[0], ground_truth.latitude.iloc[-1]))
    ground_truth_lngf = spi.interp1d(x = ground_truth_df.ts, y = ground_truth.longitude, bounds_error=False, 
                                     fill_value=(ground_truth.longitude.iloc[0], ground_truth.longitude.iloc[-1]))
    ts_new = np.arange(int(start_ts), int(end_ts - 290), 1)
    fmt_time_new = map(lambda ts: pydt.datetime.fromtimestamp(ts).isoformat(), ts_new)
    interp_df = pd.DataFrame({"ts": ts_new, "fmt_time": fmt_time_new,
                            "android3_lat": android3_latf(ts_new), "android3_lng": android3_lngf(ts_new),
                              "android4_lat": android4_latf(ts_new), "android4_lng": android4_lngf(ts_new),
                              "ios3_lat": ios3_latf(ts_new),"ios3_lng": ios3_lngf(ts_new),
                            "ios4_lat": ios4_latf(ts_new), "ios4_lng": ios4_lngf(ts_new), 
                              "ground_truth_lat":ground_truth_latf(ts_new),"ground_truth_lng":ground_truth_lngf(ts_new) },
                             index=ts_new)
    baseline_distances_1 = np.abs(interp_df.apply(calDistance1, axis=1))
    baseline_distances_2 = np.abs(interp_df.apply(calDistance2, axis=1))
    baseline_distances_3 = np.abs(interp_df.apply(calDistance3, axis=1))
    baseline_distances_4 = np.abs(interp_df.apply(calDistance4, axis=1))

    interp_df["mean_lat"] = interp_df.apply(meanLat, axis=1)
    interp_df["mean_lng"] = interp_df.apply(meanLng, axis=1)
    
    interp_df["mean_lat1"] = interp_df.apply(meanLat1, axis=1)
    interp_df["mean_lat2"] = interp_df.apply(meanLat2, axis=1)
    interp_df["mean_lat3"] = interp_df.apply(meanLat3, axis=1)
    interp_df["mean_lat4"] = interp_df.apply(meanLat4, axis=1)
    
    interp_df["mean_lng1"] = interp_df.apply(meanLng1, axis=1)
    interp_df["mean_lng2"] = interp_df.apply(meanLng2, axis=1)
    interp_df["mean_lng3"] = interp_df.apply(meanLng3, axis=1)
    interp_df["mean_lng4"] = interp_df.apply(meanLng4, axis=1)
    
    interp_df["baseline_distances1"] = baseline_distances_1
    interp_df["baseline_distances2"] = baseline_distances_2
    interp_df["baseline_distances3"] = baseline_distances_3
    interp_df["baseline_distances4"] = baseline_distances_4
    
    interp_df["ground_distances1"] = np.abs(interp_df.apply(groundDistance1, axis=1))
    interp_df["ground_distances2"] = np.abs(interp_df.apply(groundDistance2, axis=1))
    interp_df["ground_distances3"] = np.abs(interp_df.apply(groundDistance3, axis=1))
    interp_df["ground_distances4"] = np.abs(interp_df.apply(groundDistance4, axis=1))
    
    interp_df["ground_distances"] = np.abs(interp_df.apply(groundDistance, axis=1))
    return interp_df

In [ ]:
def calDistance1(row):
#    print("Calculating distance between %s,%s,%s,%s" % 
#          (row["android3_lat"], row["android3_lng"], row["android4_lat"], row["android4_lng"]))
    return pf.calDistance(ad.AttrDict({"latitude": row["android3_lat"], "longitude": row["android3_lng"]}),
                          ad.AttrDict({"latitude": row["android4_lat"], "longitude": row["android4_lng"]}))

In [ ]:
def calDistance2(row):
    return pf.calDistance(ad.AttrDict({"latitude": row["ios3_lat"], "longitude": row["ios3_lng"]}),
                          ad.AttrDict({"latitude": row["ios4_lat"], "longitude": row["ios4_lng"]}))

In [ ]:
def calDistance3(row):
    return pf.calDistance(ad.AttrDict({"latitude": row["ios3_lat"], "longitude": row["ios3_lng"]}),
                          ad.AttrDict({"latitude": row["android3_lat"], "longitude": row["android3_lng"]}))

In [ ]:
def calDistance4(row):
    return pf.calDistance(ad.AttrDict({"latitude": row["ios4_lat"], "longitude": row["ios4_lng"]}),
                          ad.AttrDict({"latitude": row["android4_lat"], "longitude": row["android4_lng"]}))

In [ ]:
def groundDistance(row):
    print "calculating distances for row %s " % row
    return pf.calDistance(ad.AttrDict({"latitude": row["ground_truth_lat"], "longitude": row["ground_truth_lng"]}),
                          ad.AttrDict({"latitude": row["mean_lat"], "longitude": row["mean_lng"]}))

In [ ]:
def groundDistance1(row):
    print "calculating distances for row %s " % row
    return pf.calDistance(ad.AttrDict({"latitude": row["ground_truth_lat"], "longitude": row["ground_truth_lng"]}),
                          ad.AttrDict({"latitude": row["mean_lat1"], "longitude": row["mean_lng1"]}))

In [ ]:
def groundDistance2(row):
    print "calculating distances for row %s " % row
    return pf.calDistance(ad.AttrDict({"latitude": row["ground_truth_lat"], "longitude": row["ground_truth_lng"]}),
                          ad.AttrDict({"latitude": row["mean_lat2"], "longitude": row["mean_lng2"]}))

In [ ]:
def groundDistance3(row):
    print "calculating distances for row %s " % row
    return pf.calDistance(ad.AttrDict({"latitude": row["ground_truth_lat"], "longitude": row["ground_truth_lng"]}),
                          ad.AttrDict({"latitude": row["mean_lat3"], "longitude": row["mean_lng3"]}))

In [ ]:
def groundDistance4(row):
    print "calculating distances for row %s " % row
    return pf.calDistance(ad.AttrDict({"latitude": row["ground_truth_lat"], "longitude": row["ground_truth_lng"]}),
                          ad.AttrDict({"latitude": row["mean_lat4"], "longitude": row["mean_lng4"]}))

In [ ]:
def meanLat1(row):
    return np.mean([row["android3_lat"], row["android4_lat"]])

In [ ]:
def meanLat2(row):
    return np.mean([row["ios3_lat"], row["ios4_lat"]])    

In [ ]:
def meanLat3(row):
    return np.mean([row["ios3_lat"],row["android3_lat"]])

In [ ]:
def meanLat4(row):
    return np.mean([row["ios4_lat"],row["android4_lat"]])

In [ ]:
def meanLat(row):
    return np.mean([row["android3_lat"], row["android4_lat"],
                    row["ios3_lat"], row["ios4_lat"]])

In [ ]:
def meanLng1(row):
    return np.mean([row["android3_lng"], row["android4_lng"]])

In [ ]:
def meanLng2(row):
    return np.mean([row["ios3_lng"], row["ios4_lng"]])

In [ ]:
def meanLng3(row):
    return np.mean([row["ios3_lng"],row["android3_lng"]])

In [ ]:
def meanLng4(row):
    return np.mean([row["ios4_lng"],row["android4_lng"]])

In [ ]:
def meanLng(row):
    return np.mean([row["android3_lng"], row["android4_lng"],
                   row["ios3_lng"], row["ios4_lng"]])

# Plotting non-interpolated vs. interpolated points 

Ground Truth Raw Latitude points against interpolated Latitude points

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
ground_truth.plot(x = 'ts',y = 'latitude', ax=axes, style='bo')
ts_new = np.arange(int(from_ts), int(end_ts), 1)
ground_truth_latf = spi.interp1d(x = ground_truth.ts, y = ground_truth.latitude, bounds_error=False, 
                                     fill_value=(ground_truth.latitude.iloc[0], ground_truth.latitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ground_truth_latf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

Same, but Longitude points

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
ground_truth.plot(x = 'ts',y = 'longitude', ax=axes, style='bo')
ground_truth_lngf = spi.interp1d(x = ground_truth.ts, y = ground_truth.longitude, bounds_error=False, 
                                     fill_value=(ground_truth.longitude.iloc[0], ground_truth.longitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ground_truth_lngf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#ios3_Latitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_iphone_df_3.plot(x = 'ts',y = 'latitude', ax=axes, style='bo')
ios3_latf = spi.interp1d(x = new_iphone_df_3.ts, y = new_iphone_df_3.latitude, bounds_error=False, 
                                     fill_value=(new_iphone_df_3.latitude.iloc[0], new_iphone_df_3.latitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ios3_latf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#ios3_Longitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_iphone_df_3.plot(x = 'ts',y = 'longitude', ax=axes, style='bo')
ios3_lngf = spi.interp1d(x = new_iphone_df_3.ts, y = new_iphone_df_3.longitude, bounds_error=False, 
                                     fill_value=(new_iphone_df_3.longitude.iloc[0], new_iphone_df_3.longitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ios3_lngf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#ios4_Latitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_iphone_df_4.plot(x = 'ts',y = 'latitude', ax=axes, style='bo')
ios4_latf = spi.interp1d(x = new_iphone_df_4.ts, y = new_iphone_df_4.latitude, bounds_error=False, 
                                     fill_value=(new_iphone_df_4.latitude.iloc[0], new_iphone_df_4.latitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ios4_latf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#ios4_longitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_iphone_df_4.plot(x = 'ts',y = 'longitude', ax=axes, style='bo')
ios4_lngf = spi.interp1d(x = new_iphone_df_4.ts, y = new_iphone_df_4.longitude, bounds_error=False, 
                                     fill_value=(new_iphone_df_4.longitude.iloc[0], new_iphone_df_4.longitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": ios4_lngf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#android3_Latitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_android_df_3.plot(x = 'ts',y = 'latitude', ax=axes, style='bo')
android3_latf = spi.interp1d(x = new_android_df_3.ts, y = new_android_df_3.latitude, bounds_error=False, 
                                     fill_value=(new_android_df_3.latitude.iloc[0], new_android_df_3.latitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": android3_latf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#android3_Longitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_android_df_3.plot(x = 'ts',y = 'longitude', ax=axes, style='bo')
android3_lngf = spi.interp1d(x = new_android_df_3.ts, y = new_android_df_3.longitude, bounds_error=False, 
                                     fill_value=(new_android_df_3.longitude.iloc[0], new_android_df_3.longitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": android3_lngf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#android4_Latitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_android_df_4.plot(x = 'ts',y = 'latitude', ax=axes, style='bo')
android4_latf = spi.interp1d(x = new_android_df_4.ts, y = new_android_df_4.latitude, bounds_error=False, 
                                     fill_value=(new_android_df_4.latitude.iloc[0], new_android_df_4.latitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": android4_latf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

In [ ]:
#android4_Longitude
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(20,5))
new_android_df_4.plot(x = 'ts',y = 'longitude', ax=axes, style='bo')
android4_lngf = spi.interp1d(x = new_android_df_4.ts, y = new_android_df_4.longitude, bounds_error=False, 
                                     fill_value=(new_android_df_4.longitude.iloc[0], new_android_df_4.longitude.iloc[-1]))
pd.DataFrame({"ts": ts_new, "gtl": android4_lngf(ts_new)}).plot(x = 'ts', y = 'gtl', ax=axes, style='r+')

# Analysing Inteprolated Values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
distances = get_interp_distance_df(android_ts_3[0],android_ts_4[0],
                                   iphone_ts_3[0], iphone_ts_4[0], ground_truth, from_ts, end_ts)

Plotting different distance - differences alongside each other

In [ ]:
distances.plot(x = "ts", y = "baseline_distances2", ax = axes)
distances.plot(x = "ts", y = "baseline_distances3", ax = axes)
distances.plot(x = "ts", y = "baseline_distances4", ax = axes)

distances.plot(x="ts", y="ground_distances2", ax = axes)
distances.plot(x="ts", y="ground_distances3", ax = axes)
distances.plot(x="ts", y="ground_distances4", ax = axes)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
distances.plot(x = "ts", y = "baseline_distances4", ax = axes)
distances.plot(x="ts", y="ground_distances4", ax = axes)

In [ ]:
import scipy.stats as spis

In [ ]:
#Example: Baseline Distances 3
mu, sigma = 15.36, 21.42 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
count, bins, ignored = plt.hist(s, 30, normed=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='r')
plt.show()

Android3 vs. Android4

In [ ]:
distances.baseline_distances1.hist()

In [ ]:
distances.ground_distances.hist()

In [ ]:
def distance(p, ground):
    return abs(p - ground)

In [ ]:
Quantity = []
d = {}
distances1 = distances[["fmt_time","ts", "baseline_distances1", 
                       "ground_distances1"]].copy()
Threshold = sorted([13,12,11,10, 9, 8, 7, 6, 5, 4, 3])
Threshold_range = [4,5,6,7,8,9,10,11,12,13]
for T in Threshold:
    d["f_dist{0}".format(T)] = distances1[(distances1.baseline_distances1 < T)]


In [ ]:
Quan = []
ground_distance_mean = []
ground_distance_sum = []
Threshold = sorted([13,12,11,10, 9, 8, 7, 6, 5, 4, 3])
for T in Threshold:
    Quan.append(d["f_dist{0}".format(T)].baseline_distances1.describe()["count"])
    ground_distance_sum.append(d["f_dist{0}".format(T)]["ground_distances1"].sum())
    ground_distance_mean.append(d["f_dist{0}".format(T)]["ground_distances1"].mean())
final_df = pd.DataFrame({"Threshold":Threshold, "Quantity":Quan,"Ground_distances_sum":ground_distance_sum, "Ground_distance_mean":ground_distance_mean})
final_df

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Threshold", y="Quantity", ax=ax)
final_df.plot(x="Threshold", secondary_y="Error", ax=ax)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Error", y="Quantity", ax=ax)

ios3 vs. ios4

In [ ]:
distances.baseline_distances2.hist()

In [ ]:
d = {}
distances2 = distances[["fmt_time","ts", "mean_lat", "mean_lng", "baseline_distances2", 
                       "ground_distances"]].copy()
Threshold = sorted([45, 40 ,35, 30,25, 20, 15, 10, 5])
for T in Threshold:
    d["f_dist{0}".format(T)] = distances2[distances2.baseline_distances2 < T]

Quan = []
error = []
for T in Threshold:
    Quan.append(d["f_dist{0}".format(T)].baseline_distances2.describe()["count"])
    error.append(d["f_dist{0}".format(T)]["ground_distances"].sum())
final_df = pd.DataFrame({"Threshold":Threshold, "Quantity":Quan, "Error":error})
final_df

fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Threshold", y="Quantity", ax=ax)
final_df.plot(x="Threshold", secondary_y="Error", ax=ax)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Error", y="Quantity", ax=ax)

ios3 vs. android3

In [ ]:
distances.baseline_distances3.hist()

In [ ]:
d = {}
distances3 = distances[["fmt_time","ts", "mean_lat", "mean_lng", "baseline_distances3", 
                       "ground_distances"]].copy()
Threshold = sorted([18, 16, 14, 12, 10, 8, 6, 4, 2])
for T in Threshold:
    d["f_dist{0}".format(T)] = distances3[distances3.baseline_distances3 < T]
Quan = []
error = []
for T in Threshold:
    Quan.append(d["f_dist{0}".format(T)].baseline_distances3.describe()["count"])
    error.append(d["f_dist{0}".format(T)]["ground_distances"].sum())
final_df = pd.DataFrame({"Threshold":Threshold, "Quantity":Quan, "Error":error})

fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Threshold", y="Error", ax=ax)
final_df.plot(x="Threshold", secondary_y="Error", ax=ax)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Error", y="Quantity", ax=ax)

ios4 vs android4

In [ ]:
distances.baseline_distances4.hist()

In [ ]:
d = {}
distances4 = distances[["fmt_time","ts", "mean_lat", "mean_lng", "baseline_distances4", 
                       "ground_distances"]].copy()
Threshold = sorted([40,35,30,25,20,15,12,10,8,5,3 ])
for T in Threshold:
    d["f_dist{0}".format(T)] = distances4[distances4.baseline_distances4 < T]
Quan = []
error = []
for T in Threshold:
    Th.append(T)
    Quan.append(d["f_dist{0}".format(T)].baseline_distances4.describe()["count"])
    error.append(d["f_dist{0}".format(T)]["ground_distances"].sum())
final_df = pd.DataFrame({"Threshold":Threshold, "Quantity":Quan, "Error":error})


#distances4.baseline_distances4.quantile([0.85, 0.90, 0.95, 0.99])
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Threshold", y="Error", ax=ax)
final_df.plot(x="Threshold", secondary_y="Error", ax=ax)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
final_df.plot(x="Error", y="Quantity", ax=ax)

# Generic Thresholds

In [ ]:
#Assume thresholds are percentiles in the range [0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99]
#for each of the four baselines, show quantity vs error, at those thresholds in one graph
# are they similar or different? compare to thresholds from previous grahs where ground distance aka error
#is taken into account. 

Thresholds = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99]
#distances1, distances2, distances3, distances4
fig, axes = plt.subplots(nrows=1, ncols=1)
q1 = distances1.baseline_distances1.quantile(Thresholds)
q2 = distances2.baseline_distances2.quantile(Thresholds)
q3 = distances3.baseline_distances3.quantile(Thresholds)
q4 = distances4.baseline_distances4.quantile(Thresholds)
q1.plot(ax = axes, style='o-')
q2.plot(ax = axes, style ='o-')
q3.plot(ax = axes, style ='o-')
q4.plot(ax = axes, style ='o-')
lines, labels = axes.get_legend_handles_labels()
axes.legend(lines, ["and3_vs_and4", "ios3_vs_ios4", "and3_vs_ios3", "and4_vs_ios4"], loc='best')

# Analysis of thresholds

In [ ]:
q2

In [ ]:
#q1 = 0.90  - 7.8
#q2 = 0.85 = 12
#q3 = 0.90 = 8.47
#q4 = 0.85 = 11.9

#Values from knee graphs
#q1 = 6 and 8  
#q2 = 10 - 15  
#q3 = 6 - 7  
#q4 = 10 - 15 ( 12.5) 


# Maps

In [ ]:
gt_gj = [{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [[37.8705228,-122.2659393], [37.8696579, -122.2659628], 
        [37.8686967, -122.2663615],
        [37.8677596, -122.2659781], [37.8681473, -122.2636508],
        [37.8672381, -122.2634622], [37.8674299, -122.2612558], 
        [37.8666323, -122.261009], [37.8669238, -122.2587255]]
  }
}]


In [ ]:
import emission.analysis.plotting.leaflet_osm.our_plotter as lo
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy

In [ ]:
    ipy.inline_maps([lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(iphone_df_3)]) + 
                    lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(iphone_df_4)]) + 
                     lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(android_df_3)]) +
                     lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(android_df_4)])])

In [ ]:
import geojson as gj

In [ ]:
gfc.get_feature_list_from_df(iphone_df_3)["features"][1]

In [ ]:
distances_pts = distances[["android3_lat","android3_lng", "android4_lat",
                           "android4_lng","ground_truth_lat","ground_truth_lng","ts",
                           "fmt_time", "ground_distances", "baseline_distances3"]].copy()

In [ ]:
gd_25 = distances_pts[(distances_pts.ground_distances < 25) & (distances_pts.ground_distances > 20)]
new_gd_25 = pd.DataFrame({"ts": gd_25.ts, "latitude": gd_25.ground_truth_lat, "longitude":gd_25.ground_truth_lng,
                         "fmt_time":gd_25.fmt_time})
and3 = pd.DataFrame({"ts": gd_25.ts, "latitude": gd_25.android3_lat, "longitude":gd_25.android3_lng, 
                    "fmt_time":gd_25.fmt_time})
and4 = pd.DataFrame({"ts": gd_25.ts, "latitude": gd_25.android4_lat, "longitude":gd_25.android4_lng,
                    "fmt_time":gd_25.fmt_time})

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(new_gd_25)]) + 
                    lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(and3)]) + 
                     lo.get_maps_for_geojson_unsectioned(
                [gfc.get_feature_list_from_df(and4)])])

In [ ]:
# reason for not using mean ( these particular points lower mean error instead of increaing it)
# calculate mean with and without points  - actual ground distances. 

In [ ]:
points_20_25 = gd_25
e_p = [points_20_25.baseline_distances3.describe()["count"],points_20_25["ground_distances"].mean(),points_20_25["ground_distances"].sum()]
e_p

In [ ]:
# linear programming? - to calculate thresholds given only baseline input data. 
# objective function: want to chose a threshold that maximises the quantity and reduces the error
# f(baseline data) threshold
# threshold - value that maximises quantity and minimizes error - but we don't have that because 
# we calculate error based on ground-distance - we can calculate quantity for given thresholds
# but need to figure out how to evaluate the quality of the points within that threshold only given baseline data

# if i'm eye - balling  - all the ideal thresholds seem to be the ones that include all the points
# based on the fact that the error is the standard mean error. the lowest error occurs at the highest threshold
# so maybe thats not a good evaluation of quality? 

# quality  - 

def threshold_calc(data):
    min_dist = min (data)
    max_dist = max(data)
    threshold_range = np.arange(min_dist,max_dist,5)
    